## Image Export for Ilastik Training

Export 5 random images from each experiment (as max-z BF projections) for annotation and classifier training.

In [1]:
%matplotlib inline
%load_ext dotenv
%dotenv env.sh
%run source/utils.py
import matplotlib
import matplotlib.pyplot as plt
from skimage import exposure
import pandas as pd
import numpy as np
import cytokit
from cytokit.image.ops import describe
import os
import os.path as osp
import tqdm

In [2]:
df = pd.read_csv('experiments.csv')
df.head()

,name,conf,dir,grid,n_ch,n_tiles,n_z,cond,rep,z_pitch
0,0000kT-0uM-np-XY01-1,experiment.yaml,noTcells,XY01,4,33,9,0000kT-0uM-np,1,200
1,0000kT-0uM-np-XY02-2,experiment.yaml,noTcells,XY02,4,33,8,0000kT-0uM-np,2,200
2,0000kT-0uM-np-XY03-3,experiment.yaml,noTcells,XY03,4,33,8,0000kT-0uM-np,3,200
3,0000kT-4uM-np-XY01-1,experiment.yaml,4uMsta,XY01,4,33,7,0000kT-4uM-np,1,200
4,0000kT-4uM-np-XY02-2,experiment.yaml,4uMsta,XY02,4,33,10,0000kT-4uM-np,2,200


In [3]:
export_dir = '/lab/data/spheroid/ilastik/mc38/training/20x-s25-spheroid-body-03/bf-maxz'
export_dir

'/lab/data/spheroid/ilastik/mc38/training/20x-s25-spheroid-body-03/bf-maxz'

In [4]:
np.random.seed(0)

def export_images(exp_name, images, export_dir, n_images=5):
    idx = np.random.choice(np.arange(len(images)), size=n_images, replace=False)
    images = [images[i] for i in idx]
    for img, ti in images:
        # Target size is 25% in both dimensions
        img = transform.rescale(img, .25, anti_aliasing=True, multichannel=False, mode='constant', preserve_range=True)
        path = osp.join(export_dir, '{}-T{:03d}-X{:03d}-Y{:03d}.tif'.format(exp_name, ti.tile_index+1, ti.tile_x+1, ti.tile_y+1))
        if not osp.exists(osp.dirname(path)):
            os.makedirs(osp.dirname(path))
        assert 0 <= img.min() <= img.max() <= 1
        img = exposure.rescale_intensity(img, in_range=(0, 1), out_range=np.uint8).astype(np.uint8)
        # Save image with original contrast, but downscaled to 25% (50% here and 50% in the original acquisition) with 8-bit depth
        ck_io.save_image(path, img)

for i, r in tqdm.tqdm(df.iterrows(), total=len(df)):
    config = get_config(r)
    raw_dir = osp.join(os.environ['EXP_GROUP_RAW_DIR'], r['dir'], r['grid'])
    # images = list(get_channel_maxz_images(config, raw_dir, 'HOECHST'))
    images = list(get_channel_maxz_images(config, raw_dir, 'BF'))
    export_images(r['name'], images, export_dir)

100%|██████████| 35/35 [1:31:03<00:00, 159.55s/it]
